In [27]:
import pandas as pd
import mysql.connector as sql
import json
from sqlalchemy import create_engine

In [2]:
interactions = pd.read_csv('./data/RAW_interactions.csv', parse_dates=[2])
users = pd.read_csv('./data/PP_users.csv', usecols=['u', 'techniques', 'items', 'ratings'])
recipes = pd.read_csv('./data/RAW_recipes.csv', parse_dates=[4], index_col='id')

In [3]:
# get id of the users
interactions_mapped_test = pd.read_csv('./data/interactions_test.csv', usecols=['user_id', 'u'])
interactions_mapped_train = pd.read_csv('./data/interactions_train.csv', usecols=['user_id', 'u'])
interactions_mapped_val = pd.read_csv('./data/interactions_validation.csv', usecols=['user_id', 'u'])

mapped_users = pd.concat([interactions_mapped_test, interactions_mapped_train, interactions_mapped_val])
mapped_users.drop_duplicates(inplace=True)

In [4]:
# add user_id to users df
get_user_id = lambda u: mapped_users.user_id[mapped_users.u == u].values[0]

users['id'] = users.u.apply(get_user_id)
users.set_index('id', inplace=True)
users.drop(['u'], axis=1, inplace=True)

In [11]:
users.head()

,techniques,items,ratings
id,,,
38094,"[8, 0, 0, 5, 6, 0, 0, 1, 0, 9, 1, 0, 0, 0, 1, ...","[1118, 27680, 32541, 137353, 16428, 28815, 658...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 4.0, ..."
1293707,"[11, 0, 0, 2, 12, 0, 0, 0, 0, 14, 5, 0, 0, 0, ...","[122140, 77036, 156817, 76957, 68818, 155600, ...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ..."
8937,"[13, 0, 0, 7, 5, 0, 1, 2, 1, 11, 0, 1, 0, 0, 1...","[168054, 87218, 35731, 1, 20475, 9039, 124834,...","[3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 5.0, ..."
126440,"[498, 13, 4, 218, 376, 3, 2, 33, 16, 591, 10, ...","[163193, 156352, 102888, 19914, 169438, 55772,...","[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 5.0, 5.0, ..."
57222,"[161, 1, 1, 86, 93, 0, 0, 11, 2, 141, 0, 16, 0...","[72857, 38652, 160427, 55772, 119999, 141777, ...","[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 4.0, 5.0, ..."


In [12]:
interactions.head()

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."


In [13]:
recipes.head()

,name,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
id,,,,,,,,,,,
137739,arriba baked winter squash mexican style,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
31490,a bit different breakfast pizza,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
112140,all in the kitchen chili,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
59389,alouette potatoes,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
44061,amish tomato ketchup for canning,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [14]:
# convert string to array
#recipe = recipes.iloc[23222]
#tags_list = [x[1:-1] for x in recipe.tags[1:-1].split(', ')]
#tags_list

# REMOVE NAN ROWS

In [15]:
interactions =  interactions.dropna()
recipes = recipes.dropna()
users = users.dropna()

In [16]:
# GET VALID CROSS DATA
interactions_rel = interactions[interactions.user_id.isin(users.index) & interactions.recipe_id.isin(recipes.index)]

In [23]:
recipes.loc[recipes.index == 85009]

,name,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
id,,,,,,,,,,,
85009,baked potato toppings,10,64342,2004-02-25,"['15-minutes-or-less', 'time-to-make', 'course...","[2786.2, 342.0, 134.0, 290.0, 161.0, 301.0, 42.0]",3,['pick whichever topping you want to use and c...,these toppings sure makes a nice change from p...,"['mayonnaise', 'salsa', 'cheddar cheese', 'ref...",13


In [53]:
with open('config.json') as json_config:
    config = json.load(json_config)

In [54]:
connection_str = 'mysql://{user}:{password}@{hostname}/{db_name}'.format(
    user = config['user_name'],
    password = config['password'],
    hostname = config['hostname'],
    db_name = config['db_name'])

In [55]:
try:
    engine = create_engine(connection_str,  echo=False)
except:
    print('DB Connection Error')

In [56]:
users.to_sql('users', con=engine,  if_exists='append', index_label='id')

DataError: (MySQLdb._exceptions.DataError) (1406, "Data too long for column 'items' at row 1")
[SQL: INSERT INTO users (id, techniques, items, ratings) VALUES (%s, %s, %s, %s)]
[parameters: ((38094, '[8, 0, 0, 5, 6, 0, 0, 1, 0, 9, 1, 0, 0, 0, 1, 0, 5, 0, 0, 0, 3, 1, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 8, 0, 0, 8, 2, 2, 0, 0, 0, 3, 3, 1, 0, 5, 0, 0, 2, 1, 1, 0, 3, 1, 7, 2, 2]', '[1118, 27680, 32541, 137353, 16428, 28815, 65803, 148743, 129939, 99377, 33025, 173660, 46706, 100515, 101276, 93176, 70383, 24204, 0, 172222, 19472, 90038, 84063, 71578, 11190, 58387, 27749, 101225, 85204, 89385, 161655]', '[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 4.0, 5.0, 5.0, 4.0, 5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0]'), (1293707, '[11, 0, 0, 2, 12, 0, 0, 0, 0, 14, 5, 0, 0, 0, 0, 0, 6, 0, 0, 1, 2, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 6, 1, 1, 1, 3, 7, 0, 0, 1, 12, 5, 3, 1, 1, 0, 0, 1, 0, 3, 0, 2, 2, 4, 1, 1]', '[122140, 77036, 156817, 76957, 68818, 155600,

In [83]:
users.loc[users.ratings.str.len() == 32185]

,techniques,items,ratings
id,,,
424680,"[2508, 26, 18, 1319, 1516, 1, 5, 123, 29, 2488...","[91791, 3798, 150824, 101819, 32984, 137191, 1...","[4.0, 5.0, 4.0, 5.0, 4.0, 5.0, 5.0, 5.0, 5.0, ..."


In [91]:
interactions.loc[interactions.recipe_id == 32541]

,user_id,recipe_id,date,rating,review
1131681,15820,32541,2002-08-14,5,This recipe puts Shake-n-Bake to shame! Follow...
1131682,20371,32541,2002-10-04,5,"Excellent! The chops were tender, with a lovel..."
1131683,63630,32541,2003-01-10,5,O my goodness. These were delicious. I didnt...
1131684,54031,32541,2003-01-17,5,I must say I don't usually like anything bread...
1131685,60231,32541,2003-01-23,5,Made these last night for dinner and they were...
1131686,56498,32541,2003-01-31,5,WOW...my husband and I really enjoyed these ch...
1131687,29212,32541,2003-07-07,5,"Hey Sue, great recipe. I followed the recipe ..."
1131688,82367,32541,2003-10-23,5,I used center cut boneless chops and while the...
1131689,85873,32541,2003-10-31,4,These are very good. Tender & juicy as stated....
1131690,108291,32541,2003-11-06,4,Great tasting chops. I used only those ingred...


In [89]:
users

,techniques,items,ratings
id,,,
38094,"[8, 0, 0, 5, 6, 0, 0, 1, 0, 9, 1, 0, 0, 0, 1, ...","[1118, 27680, 32541, 137353, 16428, 28815, 658...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 4.0, ..."
1293707,"[11, 0, 0, 2, 12, 0, 0, 0, 0, 14, 5, 0, 0, 0, ...","[122140, 77036, 156817, 76957, 68818, 155600, ...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ..."
8937,"[13, 0, 0, 7, 5, 0, 1, 2, 1, 11, 0, 1, 0, 0, 1...","[168054, 87218, 35731, 1, 20475, 9039, 124834,...","[3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 5.0, ..."
126440,"[498, 13, 4, 218, 376, 3, 2, 33, 16, 591, 10, ...","[163193, 156352, 102888, 19914, 169438, 55772,...","[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 5.0, 5.0, ..."
57222,"[161, 1, 1, 86, 93, 0, 0, 11, 2, 141, 0, 16, 0...","[72857, 38652, 160427, 55772, 119999, 141777, ...","[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 4.0, 5.0, ..."
...,...,...,...
438400,"[1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[144172, 154054]","[5.0, 5.0]"
548324,"[1, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[162436, 148228]","[5.0, 5.0]"
267518,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[162681, 172034]","[5.0, 5.0]"


In [97]:
interactions.review.str.len().argmax()

1018298

In [103]:
len(recipes.loc[recipes.name.str.len().argmax()])

KeyError: 68877

In [165]:
var = 'name'

len(recipes.iloc[recipes[var].str.len().argmax()][var])

85

In [169]:
var = 'ratings'

len(users.iloc[users[var].str.len().argmax()][var])

32185

In [170]:
var = 'review'

len(interactions.iloc[interactions[var].str.len().argmax()][var])

8587